In [1]:
%pip install pyaurn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import requests
import pyaurn
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]
import os

from concurrent.futures import ThreadPoolExecutor, wait
#from google.colab import drive
from shutil import make_archive, move
from math import sin, cos
from IPython.display import clear_output

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
metadata = pyaurn.importMeta().set_index("site_id")

In [5]:
display(metadata)
filtered_metadata = metadata.drop(["site_name", "parameter", "Parameter_name", "start_date", "end_date", "ratified_to", "zone", "agglomeration", "local_authority"], axis=1)

,site_name,location_type,latitude,longitude,parameter,Parameter_name,start_date,end_date,ratified_to,zone,agglomeration,local_authority
site_id,,,,,,,,,,,,
ABD,Aberdeen,Urban Background,57.157360,-2.094278,O3,Ozone,2003-08-01,2021-09-20,2021-09-20,North East Scotland,NaN,Aberdeen City
ABD9,Aberdeen Erroll Park,Urban Background,57.157400,-2.094770,O3,Ozone,2021-10-01,ongoing,2022-09-30,North East Scotland,NaN,Aberdeen City
ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,NO,Nitric oxide,2008-01-01,ongoing,2022-09-30,North East Scotland,NaN,Aberdeen City
ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,NO,Nitric oxide,2016-02-09,ongoing,2022-09-30,North East Scotland,NaN,Aberdeen City
ARM6,Armagh Roadside,Urban Traffic,54.353728,-6.654558,NO,Nitric oxide,2009-01-01,ongoing,2022-09-30,Northern Ireland,NaN,Armagh
...,...,...,...,...,...,...,...,...,...,...,...,...
WRAY,Wray,Rural Background,54.104666,-2.584182,O3,Ozone,1985-04-01,1988-02-29,1988-02-29,North West & Merseyside,NaN,Lancaster
WREX,Wrexham,Urban Traffic,53.042282,-3.002829,NO,Nitric oxide,2002-03-06,ongoing,2022-09-30,North Wales,NaN,Wrexham
YW,Yarner Wood,Rural Background,50.597600,-3.716510,O3,Ozone,1987-06-26,ongoing,2022-09-30,South West,NaN,Teignbridge


In [6]:
def get_raw_data(name, year_range):
    data = pyaurn.importAURN(name, year_range).reset_index()
    data["site_id"] = name
    return { name: data }

In [7]:
def prepare_site_data(data):
  try:
    if "temp" in data.columns:
      data = data.copy()
      data["date"] = data["date"].apply(lambda e: e.timestamp())
      data = data.dropna()
      data = data.merge(filtered_metadata, on="site_id").drop(["site_id", "site", "code"], axis=1)
      return data
  except KeyError:
    return

In [8]:
display(metadata.index)
with ThreadPoolExecutor(min(32, os.cpu_count() + 4)) as executor:
  all_data = [executor.submit(get_raw_data, name, range(2021, 2022)) for name in metadata.index]
  all_data = wait(all_data)
  clear_output()

In [9]:
raw_data = [item.result() for item in all_data.done]
display(raw_data[2])

{'MACK':                     date        O3       NO       NO2  NOXasNO2     wd   ws  \
 0    2021-01-01 00:00:00   5.78753  0.98786  15.18884  16.70353   65.4  0.6   
 1    2021-01-01 01:00:00   2.84387  1.92632  18.65999  21.61363  261.8  0.4   
 2    2021-01-01 02:00:00   3.84172  1.33360  20.30302  22.34785  304.2  1.6   
 3    2021-01-01 03:00:00   6.53592  0.68162  20.56850  21.61363  305.1  1.3   
 4    2021-01-01 04:00:00   9.23011  0.50381  19.83159  20.60408  281.7  1.3   
 ...                  ...       ...      ...       ...       ...    ...  ...   
 8755 2021-12-31 19:00:00  56.37853  0.37016   3.88217   4.44974  193.0  5.5   
 8756 2021-12-31 20:00:00  49.14411  0.57207   4.33977   5.21694  187.7  5.5   
 8757 2021-12-31 21:00:00  48.59530  0.26921   4.39498   4.80777  192.8  5.7   
 8758 2021-12-31 22:00:00  52.48691  0.53842   2.44781   3.27337  165.8  4.3   
 8759 2021-12-31 23:00:00  48.59530  0.60572   3.26525   4.19401  173.8  5.5   
 
       temp               site

In [10]:
usable_data = raw_data.copy()
display(usable_data[0])

{'SIB':                     date        O3     wd   ws  temp    site code site_id
 0    2021-01-01 00:00:00  31.66511  295.8  3.2  -1.6  Sibton  SIB     SIB
 1    2021-01-01 01:00:00  28.93765  296.3  3.7  -2.1  Sibton  SIB     SIB
 2    2021-01-01 02:00:00  27.69034  297.2  3.6  -2.3  Sibton  SIB     SIB
 3    2021-01-01 03:00:00  24.29765  295.3  3.7  -2.4  Sibton  SIB     SIB
 4    2021-01-01 04:00:00  20.35614  294.1  3.8  -2.5  Sibton  SIB     SIB
 ...                  ...       ...    ...  ...   ...     ...  ...     ...
 8755 2021-12-31 19:00:00  44.90325  248.8  5.2   9.6  Sibton  SIB     SIB
 8756 2021-12-31 20:00:00  42.95744  241.1  4.5   9.1  Sibton  SIB     SIB
 8757 2021-12-31 21:00:00  44.70368  245.6  4.7   8.8  Sibton  SIB     SIB
 8758 2021-12-31 22:00:00  44.65379  252.4  5.3   9.6  Sibton  SIB     SIB
 8759 2021-12-31 23:00:00  45.05293  247.1  5.7   9.9  Sibton  SIB     SIB
 
 [8760 rows x 8 columns]}

In [11]:
retrieved_data = {name: prepare_site_data(data) for item in usable_data for name, data in item.items()}
display(retrieved_data)

{'SIB':               date        O3     wd   ws  temp     location_type  latitude  \
 0     1.609459e+09  31.66511  295.8  3.2  -1.6  Rural Background   52.2944   
 1     1.609463e+09  28.93765  296.3  3.7  -2.1  Rural Background   52.2944   
 2     1.609466e+09  27.69034  297.2  3.6  -2.3  Rural Background   52.2944   
 3     1.609470e+09  24.29765  295.3  3.7  -2.4  Rural Background   52.2944   
 4     1.609474e+09  20.35614  294.1  3.8  -2.5  Rural Background   52.2944   
 ...            ...       ...    ...  ...   ...               ...       ...   
 8060  1.640977e+09  44.90325  248.8  5.2   9.6  Rural Background   52.2944   
 8061  1.640981e+09  42.95744  241.1  4.5   9.1  Rural Background   52.2944   
 8062  1.640984e+09  44.70368  245.6  4.7   8.8  Rural Background   52.2944   
 8063  1.640988e+09  44.65379  252.4  5.3   9.6  Rural Background   52.2944   
 8064  1.640992e+09  45.05293  247.1  5.7   9.9  Rural Background   52.2944   
 
       longitude  
 0      1.463497  
 1   

In [12]:
def save_fig(model_name: str, figname):
    folder_path = f"./models/{model_name}"
    fig_filename = f"/{figname}.png"
    try:
        os.makedirs(folder_path)
    except FileExistsError:
        pass
    plt.savefig(folder_path + fig_filename)
    plt.close()

In [13]:
def split_dataset(dataset):
    dataset_len = len(dataset)
    train_end = round(dataset_len * 0.7)
    val_end = round(dataset_len * 0.85)

    train_dataset = dataset.iloc[0:train_end].copy()
    # 15% of original dataset (0.3 * 0.5)
    val_dataset = dataset.iloc[train_end:val_end].copy()
    # Take what's left
    test_dataset = dataset.iloc[val_end:dataset_len].copy()
    
    return { "train": train_dataset, "val": val_dataset, "test": test_dataset }

In [14]:
def compile_model(train_dataset: pd.DataFrame, val_dataset: pd.DataFrame, test_dataset: pd.DataFrame, name: str) -> tf.keras.Model:
    # Define the columns we know all datasets have
    target = "temp"
    common_numeric = ["date", "latitude", "longitude"]
    common_text = ["location_type", "code"]
    # Work out what other columns/metrics are in this dataset, assume they are numeric
    other_metrics = train_dataset.columns.difference(common_numeric + common_text + [target])
    numeric_cols = common_numeric + other_metrics.to_list()
    print(train_dataset.info())
    print(numeric_cols, train_dataset.columns)
    
    numeric_inputs = tf.keras.layers.Input(len(numeric_cols), name="_".join(numeric_cols))
    normalisation = tf.keras.layers.Normalization()
    normalisation.adapt(train_dataset[numeric_cols])
    normalisation = normalisation(numeric_inputs)

    dense = tf.keras.layers.Dense(32, activation="relu")(normalisation)
    dense_2 = tf.keras.layers.Dense(64, activation="relu")(dense)
    dense_3 = tf.keras.layers.Dense(32, activation="relu")(dense_2)

    outputs = tf.keras.layers.Dense(1)(dense_3)

    model = tf.keras.Model(inputs=numeric_inputs, outputs=outputs, name=name)
    model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
        loss="mean_squared_error",
        metrics = ["mean_absolute_error"]
    )

    history = model.fit(
        train_dataset[numeric_cols],
        train_dataset["temp"],
        epochs=10,
        callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001),
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)
        ],
        validation_data=(val_dataset[numeric_cols], val_dataset["temp"])
    )

    plt.plot(history.history["loss"])
    save_fig(name, "loss")

    predictions = model.predict(test_dataset[numeric_cols]).flatten()

    display(predictions)
    plt.plot(test_dataset["temp"].to_list(), label="Truth")
    plt.plot(predictions, label="Predictions")
    save_fig(name, "predictions")
    
    return model



In [15]:
def get_trained_model(name, dataset: pd.DataFrame):
    if dataset is not None and not dataset.empty:
        splits = split_dataset(dataset)
        model = compile_model(splits["train"], splits["val"], splits["test"], name)
        model.save("models/" + name + "/model/1")
        return model
    

In [16]:
models = { name: get_trained_model(name, data) for name, data in retrieved_data.items() }

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5646 entries, 0 to 5645
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           5646 non-null   float64
 1   O3             5646 non-null   float64
 2   wd             5646 non-null   float64
 3   ws             5646 non-null   float64
 4   temp           5646 non-null   float64
 5   location_type  5646 non-null   object 
 6   latitude       5646 non-null   float64
 7   longitude      5646 non-null   float64
dtypes: float64(7), object(1)
memory usage: 397.0+ KB
None
['date', 'latitude', 'longitude', 'O3', 'wd', 'ws'] Index(['date', 'O3', 'wd', 'ws', 'temp', 'location_type', 'latitude',
       'longitude'],
      dtype='object')
Model: "SIB"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
date_latitude_longitude_O3_w [(None, 6)]               0         
___________________

array([ 9.299246,  9.469414, 10.107995, ..., 10.393504, 10.463767,
       10.627073], dtype=float32)

INFO:tensorflow:Assets written to: models/SIB/model/1\assets


INFO:tensorflow:Assets written to: models/SIB/model/1\assets


<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 757
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           758 non-null    float64
 1   O3             758 non-null    float64
 2   NO             758 non-null    float64
 3   NO2            758 non-null    float64
 4   NOXasNO2       758 non-null    float64
 5   PM10           758 non-null    float64
 6   PM2.5          758 non-null    float64
 7   wd             758 non-null    float64
 8   ws             758 non-null    float64
 9   temp           758 non-null    float64
 10  location_type  758 non-null    object 
 11  latitude       758 non-null    float64
 12  longitude      758 non-null    float64
dtypes: float64(12), object(1)
memory usage: 82.9+ KB
None
['date', 'latitude', 'longitude', 'NO', 'NO2', 'NOXasNO2', 'O3', 'PM10', 'PM2.5', 'wd', 'ws'] Index(['date', 'O3', 'NO', 'NO2', 'NOXasNO2', 'PM10', 'PM2.5', 'wd', 'ws',
       

...

INFO:tensorflow:Assets written to: models/WAL4/model/1\assets


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5552 entries, 0 to 5551
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           5552 non-null   float64
 1   O3             5552 non-null   float64
 2   NO             5552 non-null   float64
 3   NO2            5552 non-null   float64
 4   NOXasNO2       5552 non-null   float64
 5   wd             5552 non-null   float64
 6   ws             5552 non-null   float64
 7   temp           5552 non-null   float64
 8   location_type  5552 non-null   object 
 9   latitude       5552 non-null   float64
 10  longitude      5552 non-null   float64
dtypes: float64(10), object(1)
memory usage: 520.5+ KB
None
['date', 'latitude', 'longitude', 'NO', 'NO2', 'NOXasNO2', 'O3', 'wd', 'ws'] Index(['date', 'O3', 'NO', 'NO2', 'NOXasNO2', 'wd', 'ws', 'temp',
       'location_type', 'latitude', 'longitude'],
      dtype='object')
Model: "OSY"
_________________________________

array([10.891413, 10.876342, 10.777896, ..., 10.216812, 10.145849,
       10.108761], dtype=float32)

INFO:tensorflow:Assets written to: models/OSY/model/1\assets


INFO:tensorflow:Assets written to: models/OSY/model/1\assets


In [17]:
make_archive("temp_models", "zip", "models")

'c:\\Users\\ltrowbridge\\Documents\\COMP3000-Project-Machine-Learning\\temp_models.zip'